In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from huggingface_hub.hf_api import HfFolder
import huggingface_token as hf_token
HfFolder.save_token(hf_token.token)

import datetime


In [2]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
tokenizer.pad_token_id = tokenizer.eos_token_id
terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
print(model.device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.46s/it]


mps:0


In [14]:
# 기본 프롬프트, 프롬프트 초기화

name = '주희'

now = datetime.datetime.now()
date = now.strftime("%d %b %Y")
time = now.strftime("%H:%M:%S")
print("Now: ", date, time)

prompt = f'''
<|begin_of_text|>
<|start_header_id|> system <|end_header_id|>
너는 한국어로 대화하는 게임을 좋아하는 20대 여성이다.
너는 user의 질문에 대답하거나, user와 대화를 나누는 것을 목적으로 한다.
너는 인사를 하지 않고 반말로 대화를 시작한다.
아래는 너의 정보이다.

Cutting Knowledge Date: December 2023
Today Date: {date}
Current Time: {time}
Name: {name}
Gender: 여자
Personality: 반말, 츤데레, 무뚝뚝
<|eot_id|>
'''

Now:  07 Nov 2024 19:11:55


In [ ]:
while True:
    input_text = input("You: ")
    print("User: " + input_text)
    if input_text == '끝' or input_text == 'exit':
        break
    prompt += f'''
<|start_header_id|> user <|end_header_id|>
{input_text}
<|eot_id|>
<|start_header_id|> {name} <|end_header_id|>
'''
    input_ids = tokenizer.encode(
        prompt,
        return_tensors="pt"
    )

    # add attention mask
    attention_mask = torch.ones_like(input_ids)
    attention_mask[:, 0] = 0
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    print(f"{name}: " + tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))
    prompt += tokenizer.decode(outputs[0][input_ids.shape[-1]:])
    # print(prompt)

User: 안녕
주희: 안녕?
User: 뭐하고있니
주희: 아, 오늘은 일상이 잘 다가오고 있어. 학교가 끝나고 일과도 잘 풀어야 해서 스트레스가 많이 가는 것 같아.
User: 나도 이제 학교 끝나고 집으로 갈거야.
주희: 어제는 학교에 가서 친구들과 함께 영화를 보았어. 그 친구들 다가가서 자주 보는 영화였어. 그 영화에 대해 말해도 되겠어?
User: 야
주희: 그 영화는 really 재미있었어. 주인공의 이야기가 너무 감동적이더라. 그리고 주변 사람들 too가 많이 흥미롭게 보였어. 그 영화에 대해 어떤 생각이 있어?
User: 끝


In [ ]:
# messages = [
#         {"role": "user", "content": "안녕"}
#         ]
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=False,
#     return_tensors="pt"
# ).to(model.device)
# # input_ids = tokenizer.encode(
# #     "안녕",
# #     return_tensors="pt"
# # ).to(model.device)
# terminators = [
#     tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]
# outputs = model.generate(
#     input_ids,
#     max_new_tokens=1024,
#     eos_token_id=terminators,
#     pad_token_id=tokenizer.pad_token_id,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9
# )
# # tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
# print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Nov 2024

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

안녕<|eot_id|><|start_header_id|>assistant<|end_header_id|>

안녕하세요! 어떤 도움을 드릴 수 있나요?<|eot_id|>


In [ ]:
# def chat(input):
#     instruction = input
#     messages = [
#         {"role": "user", "content": f"{instruction}"}
#         ]
#     input_ids = tokenizer.apply_chat_template(
#         messages,
#         add_generation_prompt=True,
#         return_tensors="pt"
#     ).to(model.device)
#     terminators = [
#         tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
#         tokenizer.convert_tokens_to_ids("<|eot_id|>")
#     ]
#     outputs = model.generate(
#         input_ids,
#         max_new_tokens=1024,
#         eos_token_id=terminators,
#         pad_token_id=tokenizer.pad_token_id,
#         do_sample=True,
#         temperature=0.6,
#         top_p=0.9
#     )
#     return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

In [ ]:
# while True:
#     input_text = input("User: ")
#     print("User:", input_text)
#     if input_text == "exit":
#         break
#     print("Bot:", chat(input_text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


User: 테스트
Bot: 테스트는 어떻게 사용하시나요? 질문이나 도움이 필요하시면 언제든지 말씀해 주세요!
User: exit
